In [2]:
from google import genai
import os
from google.genai import types

In [3]:
def llm(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt
    )
    return response.text

In [4]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [5]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [6]:
es_client.info()

ObjectApiResponse({'name': '7b385051d6db', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0Wnl4O2hSFexy0_4Q1m43A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [9]:
#!pip install requests

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index_name = "course-questions-4"

es_client.indices.create(index=index_name, settings=index_settings['settings'], mappings=index_settings['mappings'])

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-4'})

In [10]:
#!pip install tqdm
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 412.86it/s]


In [13]:
def elastic_search_max_score(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, size=search_query['size'], query=search_query['query'])
    result_scores = []
    for hit in response['hits']['hits']:
        result_scores.append(hit['_score'])
    return max(result_scores)

elastic_search_max_score('How do execute a command on a Kubernetes pod?')
    

44.50556

In [14]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, size=search_query['size'], query=search_query['query'])
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs
    

In [15]:
elastic_search('How do copy a file to a Docker container?')

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [17]:
query = "How do copy a file to a Docker container?"
search_results = elastic_search(query)

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context = ""
    for doc in search_results:
        context_template = """
Q: {question}
A: {text}
""".strip()
        context = context + context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

prompt = build_prompt(query, search_results)
print(len(prompt))


1446


In [26]:
from vertexai.preview import tokenization

model_name = "gemini-1.5-flash-001"
tokenizer = tokenization.get_tokenizer_for_model(model_name)

result = tokenizer.compute_tokens(prompt)
len_result = tokenizer.count_tokens(prompt)

print(f"Token count: {len_result}")
print(f"Encoded tokens: {result}")

Token count: CountTokensResult(total_tokens=353)
Encoded tokens: PreviewComputeTokensResult(tokens_info=[TokensInfo(token_ids=[2045, 235303, 478, 476, 3205, 10649, 20409, 235265, 10358, 573, 58470, 3482, 611, 573, 148990, 774, 573, 22217, 8746, 235265, 108, 7056, 1297, 573, 10275, 774, 573, 148990, 1185, 39534, 573, 58470, 235265, 109, 53118, 235292, 2250, 749, 5469, 476, 2482, 577, 476, 60541, 11254, 235336, 109, 71157, 235292, 108, 235368, 235292, 2250, 749, 590, 24391, 476, 54742, 11254, 235336, 108, 235280, 235292, 37181, 573, 11254, 2416, 575, 29295, 4058, 578, 135522, 573, 7929, 3371, 235269, 712, 674, 665, 11207, 476, 41912, 5017, 235265, 108, 38584, 2060, 728, 500, 3297, 9203, 3371, 41912, 968, 2502, 235313, 108, 2495, 573, 11254, 603, 3303, 5327, 235269, 17991, 476, 5017, 575, 573, 3724, 11254, 235292, 108, 38584, 9516, 591, 3655, 573, 11254, 235290, 539, 235275, 108, 38584, 15287, 728, 500, 968, 7139, 235290, 539, 235313, 41912, 108, 235278, 107749, 595, 37661, 235275, 109, 2